## 1. Import libraries

In [92]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import sklearn
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## 2. Data collection

In [95]:
# define url
url = 'https://en.wikipedia.org/wiki/Passengers_of_the_Titanic'
response = requests.get(url)

# define the attributes for data
Name = []
Age = []
Hometown = []
Boarded = []
Destination = []
Lifeboat = []
Class = []
Survive = []

# check if the url is ok or not
if (response.status_code != 200):
    print('Error')
else:
    url_data = response.text
    soup = BeautifulSoup(url_data, 'html.parser')

    # find tags data
    passenger_tags = soup.find_all('table', {'class':'wikitable sortable'})
    clss = 0
    span_cnt = np.full(6, 0, dtype = int)
    # each passenger_tag is passengers with each class
    for passenger_tag in passenger_tags:
        tr_tags = passenger_tag.find_all('tr')
        
        clss += 1
        check = np.full(6, True, dtype = bool)
        cnt = 0
        for tr_tag in tr_tags:
            # each tr_tag is a passenger data
            td_tags = tr_tag.find_all('td')
            
            if not td_tags:
                continue

            if tr_tag.get('style'):
                Survive.append(1)
            else:
                Survive.append(0)

            # each td_tag is a passenger attributes
            i = 0
            cnt += 1
            for td_tag in td_tags:
                
                row = int(td_tag.get('rowspan')) if td_tag.get('rowspan') else 1

                while i < 6 and check[i] == False:
                    span_cnt[i] -= 1
                    if span_cnt[i] == 0:
                        check[i] = True
                    i += 1

                
                if i >= 6:
                    break
                
                if check[i] == True:
                    span_cnt[i] += row-1
                    if span_cnt[i] > 0:
                        check[i] = False
                    
                    if i == 0:
                        Class.append(clss)
                        while row > 0:
                            Name.append(td_tag.text) if td_tag else Name.append(np.nan)
                            row -= 1
                    elif i == 1:
                        while row > 0:
                            Age.append(td_tag.text) if td_tag else Age.append(np.nan)
                            row -= 1
                    elif i == 2:
                        while row > 0:
                            Hometown.append(td_tag.text) if td_tag else Hometown.append(np.nan)
                            row -= 1
                    elif i == 3:
                        while row > 0:
                            Boarded.append(td_tag.text) if td_tag else Boarded.append(np.nan)
                            row -= 1
                    elif i == 4:
                        while row > 0:
                            Destination.append(td_tag.text) if td_tag else Destination.append(np.nan)
                            row -= 1      
                    elif i == 5:
                        while row > 0:
                            Lifeboat.append(td_tag.text) if td_tag else Lifeboat.append(np.nan)
                            row -= 1
                    i+=1
            
            while i < 6 and check[i] == False:
                    span_cnt[i] -= 1
                    if span_cnt[i] == 0:
                        check[i] = True
                    i += 1

raw_df = pd.DataFrame({'Name':Name,
                       'Age': Age,
                       'Hometown': Hometown,
                       'Boarded': Boarded,
                       'Destination': Destination,
                       'Lifeboat': Lifeboat,
                       'Class': Class,
                       'Survive': Survive})

raw_df.head(10)

,Name,Age,Hometown,Boarded,Destination,Lifeboat,Class,Survive
0,"Allen, Miss Elizabeth Walton",29,"St Louis, Missouri, US",Southampton,St Louis,\n,1,1
1,"Allison, Mr. Hudson Creighton",30,"Montreal, Quebec, Canada",Southampton,"Montreal, Quebec, Canada",,1,0
2,"and chauffeur, Mr. George Swane[71]",19,,Southampton,"Montreal, Quebec, Canada",,1,0
3,"and cook, Miss Amelia Mary ""Mildred"" Brown[71]",18,"London, England, UK",Southampton,"Montreal, Quebec, Canada",11,1,1
4,"Allison, Mrs. Bessie Waldo (née Daniels)",25,"Montreal, Quebec, Canada",Southampton,"Montreal, Quebec, Canada",,1,0
5,"and maid, Miss Sarah Daniels",33,"Montreal, Quebec, Canada",Southampton,"Montreal, Quebec, Canada",8,1,1
6,"Allison, Miss Helen Loraine",2,"Montreal, Quebec, Canada",Southampton,"Montreal, Quebec, Canada",,1,0
7,"Allison, Master Hudson Trevor",11 mo.,"Montreal, Quebec, Canada",Southampton,"Montreal, Quebec, Canada",11,1,1
8,"and nurse, Miss Alice Catherine Cleaver",22,"London, England, UK\n",Southampton,"Montreal, Quebec, Canada",11,1,1
9,"Anderson, Mr. Harry",47,New York City,Southampton,New York City,3\n,1,1
